In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install tensorflow==1.13.1 

In [ ]:
#!pip install tensorflow==1.13.1 

# ****Novel approach

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import scipy.sparse as sp
import numpy as np
from tensorflow.keras.layers import SimpleRNNCell
from tensorflow.compat.v1.nn.rnn_cell import BasicRNNCell
import tensorflow as tf
from tensorflow.python.platform import tf_logging as logging
import pickle as pkl
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import os
import numpy.linalg as la
from sklearn.metrics import mean_squared_error,mean_absolute_error
#import matplotlib.pyplot as plt
import time
time_start=time.time()

In [ ]:
# Loading of Dataset-SZ Traffic
def load_sz_data():
    sz_adj = pd.read_csv('../input/traffic/sz_adj.csv',header=None)
    adj = np.mat(sz_adj)
    sz_tf = pd.read_csv('../input/traffic/sz_speed.csv')
    return sz_tf, adj

data, adj = load_sz_data()

In [ ]:
print(data.shape)
time_len=data.shape[0] # Time sequence length
num_nodes=data.shape[1] #Number of Roads

In [ ]:
# Setting the variables
output_dim=pre_len=2
seq_len=4
num_units=100
train_rate=0.8
batch_size=32

In [ ]:
#Normalization : Traffic Speed Data

data1 =np.mat(data,dtype=np.float32)

max_value = np.max(data1)
data1  = data1/max_value
#print(data1)

   

In [ ]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate) #2976 *0.8 =2380
    train_data = data[0:train_size] #  [0:2380]
    test_data = data[train_size:time_len] #[2380:2976]
    print(train_data.shape,'----->',test_data.shape)

    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len): #(2380-4-1)=2375
        a = train_data[i: i + seq_len + pre_len] #[0:0+4+1] 
        trainX.append(a[0 : seq_len]) #a[0:4] 4 time * 156 roads
        trainY.append(a[seq_len : seq_len + pre_len]) #a[4:4+1] 1 time*156 
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
      
    trainX1 = np.array(trainX) 
    trainY1 = np.array(trainY)
    testX1 = np.array(testX)
    testY1 = np.array(testY)
    return trainX1, trainY1, testX1, testY1

trainX, trainY, testX, testY = preprocess_data(data1, time_len, train_rate, seq_len, pre_len)

totalbatch = int(trainX.shape[0]/batch_size)
training_data_count = len(trainX)  
print('Train Test Split Details :')
print('Train x ----> ',len(trainX))
print('Train y ----> ',len(trainY))
print(trainX.shape)
print('Test x ----> ',len(testX))
print('Test y ----> ',len(testY))
print('Total Batch ----> ',totalbatch)
#print('\nTrain Sample Details :')
#print(trainX[0],'--->',trainY[0])


In [ ]:
def weight_variable_glorot(input_dim, output_dim, name=""):
        init_range = np.sqrt(1 / (input_dim))
        initial = tf.random.uniform([input_dim, output_dim], minval=-init_range, maxval=init_range, dtype=tf.float32)
        return tf.Variable(initial,name=name) 
#weights =weight_variable_glorot(4, 64, name='weights')
#weights

In [ ]:
class stgcnCell(BasicRNNCell):
    """Temporal Graph Convolutional Network """

    def call(self, inputs, **kwargs):
        pass

    def __init__(self, num_units, adj, num_nodes, input_size=None,
                 act=tf.nn.tanh, reuse=None):

        super(stgcnCell, self).__init__(num_units=num_units,reuse=reuse)
        self._act = act
        self._nodes = num_nodes
        self._units = num_units
        self._adj = []
        self._adj.append(self.calculate_laplacian(adj))

    @staticmethod
    def _build_sparse_matrix(L):
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data, L.shape)
        return tf.sparse_reorder(L)

    def calculate_laplacian(self,adj, lambda_max=1):  
        adj = self.normalized_adj(adj + sp.eye(adj.shape[0])) # normalisation(self identity matrix + adj)
        adj = sp.csr_matrix(adj) #compressed sparse matrix
        adj = adj.astype(np.float32)
        return self.sparse_to_tuple(adj)
    
    def normalized_adj(self,adj):
        adj = sp.coo_matrix(adj)
        degree = np.array(adj.sum(1)) # Degree Matrix row wise sum
        d_inv_sqrt = np.power(degree, -0.5).flatten() # D inv = Degree ^-0.5 
        d_mat_inv_sqrt = sp.diags(d_inv_sqrt) #substitution of the 1D array degree in a 2D matrix diagonals
        normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo() # norm= D^-0.5 * adj * D^-0.5
        normalized_adj = normalized_adj.astype(np.float32) 
        return normalized_adj
     
    ''' def weight_variable_glorot(input_dim, output_dim, name=""):
        init_range = np.sqrt(6.0 / (input_dim + output_dim))
        initial = tf.random_uniform([input_dim, output_dim], minval=-init_range, maxval=init_range, dtype=tf.float32)
        return tf.Variable(initial,name=name) '''
    
    def sparse_to_tuple(self,mx):
        mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose() #coordinate stacking row and column wise and transpose
        L = tf.SparseTensor(coords, mx.data, mx.shape) # mx.shape= (156,156)
        #print('shape ---->',mx.shape)
        return tf.sparse.reorder(L) #row major ordering
        
    def init_state(self,batch_size):       
        state = tf.zeros(shape=[batch_size, self._num_nodes*self._num_units], dtype=tf.float32)
        return state  
               
    @staticmethod
    def _concat(x, x_):
        x_ = tf.expand_dims(x_, 0)
        return tf.concat([x, x_], axis=0)   
    @property
    def state_size(self):
        return self._nodes * self._units

    @property
    def output_size(self):
        return self._units

    def __call__(self, inputs, state, scope=None):

        with tf.compat.v1.variable_scope(scope or "tgcn",reuse=tf.compat.v1.AUTO_REUSE):
            with tf.compat.v1.variable_scope("gates",reuse=tf.compat.v1.AUTO_REUSE):  
                value = tf.nn.sigmoid(
                    self._gc(inputs, state, 2 * self._units, bias=1.0, scope=scope))
                r, u = tf.split(value=value, num_or_size_splits=2, axis=1)
            with tf.compat.v1.variable_scope("candidate",reuse=tf.compat.v1.AUTO_REUSE):
                r_state = r * state
                c = self._act(self._gc(inputs, r_state, self._units, scope=scope))
            new_h = u * state + (1 - u) * c
        return new_h, new_h


    def _gc(self, inputs, state, output_size, bias=0.0, scope=None):
        ## inputs:(-1,num_nodes)
        inputs = tf.expand_dims(inputs, 2)
        ## state:(batch,num_node,gru_units)
        state = tf.reshape(state, (-1, self._nodes, self._units))#(-1,156,64)
        ## concat
        x_s = tf.concat([inputs, state], axis=2)
        input_size = x_s.get_shape()[2]
        ## (num_node,input_size,-1)
        x0 = tf.transpose(x_s, perm=[1, 2, 0])  
        x0 = tf.reshape(x0, shape=[self._nodes, -1])
        
        scope = tf.compat.v1.get_variable_scope()
        with tf.compat.v1.variable_scope(scope):
            for m in self._adj:
                x1 = tf.compat.v1.sparse_tensor_dense_matmul(m, x0)
#                print(x1)
            x = tf.reshape(x1, shape=[self._nodes, input_size,-1])
            x = tf.transpose(x,perm=[2,0,1])
            x = tf.reshape(x, shape=[-1, input_size])
            weights = weight_variable_glorot(input_size, output_size, name='weights')
            x = tf.matmul(x, weights)  # (batch_size * self._nodes, output_size)
            biases = tf.compat.v1.get_variable(
                "biases", [output_size], initializer=tf.constant_initializer(bias))
            x = tf.nn.bias_add(x, biases)
            x = tf.reshape(x, shape=[-1, self._nodes, output_size])
            x = tf.reshape(x, shape=[-1, self._nodes * output_size])
        return x

In [ ]:
def STGCN(_X, _weights, _biases):
    ###
    cell_1 = stgcnCell(num_units, adj, num_nodes=num_nodes)
    cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([cell_1], state_is_tuple=True)
    _X = tf.unstack(_X, axis=1)
    #rnncell = tf.keras.layers.RNN(cell, return_state=True)
    #outputs, states= rnncell(_X)
    outputs, states=tf.compat.v1.nn.static_rnn(cell, _X, dtype=tf.float32)
    m = []
    for i in outputs:
        o = tf.reshape(i,shape=[-1,num_nodes,num_units])
        o = tf.reshape(o,shape=[-1,num_units])
        m.append(o)
    last_output = m[-1]
    output = tf.matmul(last_output, _weights['out']) + _biases['out']
    output = tf.reshape(output,shape=[-1,num_nodes,pre_len])
    output = tf.transpose(output, perm=[0,2,1])
    output = tf.reshape(output, shape=[-1,num_nodes])
    return output, m, states

In [ ]:
tf.compat.v1.disable_eager_execution()
inputs = tf.compat.v1.placeholder(tf.float32, shape=[None, seq_len, num_nodes])
labels = tf.compat.v1.placeholder(tf.float32, shape=[None, pre_len, num_nodes])
weights = {
    'out': tf.Variable(tf.random.normal([num_units, pre_len], mean=1.0), name='weight_o')}
biases = {
    'out': tf.Variable(tf.random.normal([pre_len]),name='bias_o')}
print(type(inputs))
pred,ttts,ttto = STGCN(inputs, weights, biases)

In [ ]:
y_pred = pred

In [ ]:
lambda_loss = 0.0015
Lreg = lambda_loss * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.compat.v1.trainable_variables())
label = tf.reshape(labels, [-1,num_nodes])

In [ ]:
loss = tf.reduce_mean(tf.nn.l2_loss(y_pred-label) + Lreg)

In [ ]:
error = tf.sqrt(tf.reduce_mean(tf.square(y_pred-label)))

In [ ]:
lr=0.001
opt = tf.compat.v1.train.AdamOptimizer(lr)
optimizer=opt.minimize(loss)

In [ ]:
variables = tf.compat.v1.global_variables()
training_epoch=1000

In [ ]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [ ]:
def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    r2 = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return rmse, mae, 1-F_norm, r2, var

In [ ]:
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_acc,test_r2,test_var,test_pred = [],[],[],[],[],[],[]

In [ ]:
def acc(a,b):  
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    train_acc=1-F_norm
    return train_acc

In [ ]:
for epoch in range(training_epoch):
    print("Epoch ", epoch)
    for m in range(totalbatch):
        mini_batch = trainX[m * batch_size : (m+1) * batch_size]
        mini_label = trainY[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)
        batch_rmse.append(rmse1 * max_value)
        train_label=np.reshape(mini_label,[-1,num_nodes])
        

     # Test completely at every epoch
    print("Accuracy ----> ", evaluation(train_label,train_output)[2])
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:testX, labels:testY})
    #train_label=np.reshape(trainY,[-1,num_nodes])
    #train_acc=acc(train_label,train_output)
    test_label = np.reshape(testY,[-1,num_nodes])
    rmse, mae, acc, r2_score, var_score = evaluation(test_label, test_output)
    test_label1 = test_label * max_value#Inverse normalization
    test_output1 = test_output * max_value
    test_loss.append(loss2)
    test_rmse.append(rmse * max_value)
    test_mae.append(mae * max_value)
    test_acc.append(acc)
    test_r2.append(r2_score)
    test_var.append(var_score)
    test_pred.append(test_output1)
    print(mini_label.shape,train_output.shape)
    print('Iter:{}'.format(epoch),
          'train_rmse:{:.4}'.format(batch_rmse[-1]),
          'test_loss:{:.4}'.format(loss2),
          'test_rmse:{:.4}'.format(rmse),
          'test_acc:{:.4}'.format(acc))
    
        
time_end = time.time()
print(time_end-time_start,'s')

In [ ]:
b = int(len(batch_rmse)/totalbatch)
batch_rmse1 = [i for i in batch_rmse]
train_rmse = [(sum(batch_rmse1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
batch_loss1 = [i for i in batch_loss]
train_loss = [(sum(batch_loss1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]

index = test_rmse.index(np.min(test_rmse))
test_result = test_pred[index]
var = pd.DataFrame(test_result)

In [ ]:
print("Testing:")
print('min_rmse:%r'%(np.min(test_rmse)),
      'min_mae:%r'%(test_mae[index]),
      'max_acc:%r'%(test_acc[index]),
      'r2:%r'%(test_r2[index]),
      'var:%r'%test_var[index])

In [ ]:
print("Training:")
rmse, mae, acc, r2_score, var_score = evaluation(train_label,train_output)
print('min_rmse:%r'%(rmse),
      'min_mae:%r'%(mae),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score))

In [ ]:
#inverse normalisation
print('min_rmse:%r'%(rmse*max_value),
      'min_mae:%r'%(mae*max_value),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score))

In [ ]:
##visualisation
##When hidden units = 8,16,32,64,128
import matplotlib.pyplot as plt 
hidden=[8,16,32,64,100,128]
mae=[2.96,2.82,2.74,2.72,2.70,2.72]
rmse=[4.15,4.04,3.96,3.95,3.91,3.93]
fig,ax=plt.subplots()
ax.plot(hidden,mae,color="red",label="mae",marker="o")
ax.set_xlabel("Hidden Units",fontsize=14)
ax.set_ylabel("MAE",color="red",fontsize=14)
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(hidden,rmse,color="blue",label="rmse",marker="o")
#ax.set_xlabel("Hidden Units",fontsize=14)
ax2.set_ylabel("RMSE",color="blue",fontsize=14)
plt.show()


In [ ]:
acc=[7.20,7.22,7.27,7.29,7.31]
r2=[0.8528,0.8531,0.8532,0.8534,0.8541,0.8538]
var=[0.852,0.855,0.8525,0.8575,0.8625,0.8575]
##visualisation
##When hidden units = 8,16,32,64,128
hidden=[8,16,32,64,100,128]
fig,ax=plt.subplots()
ax.plot(hidden,r2,color="red",label="mae",marker="o")
ax.set_xlabel("Hidden Units",fontsize=14)
ax.set_ylabel("R2",color="red",fontsize=14)
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(hidden,var,color="blue",label="rmse",marker="o")
#ax.set_xlabel("Hidden Units",fontsize=14)
ax2.set_ylabel("Variance",color="blue",fontsize=14)
plt.show()


In [ ]:
hidden=[8,16,32,64,100,128]
acc=[72.05,72.2,72.7,72.9,73.1,73]
plt.plot(hidden,acc,label="accuracy",marker="o")
plt.xlabel("Hidden Units")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# **STGCN Model**

In [ ]:
time_start=time.time()

In [ ]:
class tstgcnCell(BasicRNNCell):
    """Temporal Graph Convolutional Network """

    def call(self, inputs, **kwargs):
        pass

    def __init__(self, num_units, adj, num_nodes, input_size=None,
                 act=tf.nn.tanh, reuse=None):

        super(tstgcnCell, self).__init__(num_units=num_units,reuse=reuse)
        self._act = act
        self._nodes = num_nodes
        self._units = num_units
        self._adj = []
        self._adj.append(self.calculate_laplacian(adj))

    @staticmethod
    def _build_sparse_matrix(L):
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data, L.shape)
        return tf.sparse_reorder(L)

    def calculate_laplacian(self,adj, lambda_max=1):  
        adj = self.normalized_adj(adj + sp.eye(adj.shape[0])) # normalisation(self identity matrix + adj)
        adj = sp.csr_matrix(adj) #compressed sparse matrix
        adj = adj.astype(np.float32)
        return self.sparse_to_tuple(adj)
    
    def normalized_adj(self,adj):
        adj = sp.coo_matrix(adj)
        degree = np.array(adj.sum(1)) # Degree Matrix row wise sum
        d_inv_sqrt = np.power(degree, -0.5).flatten() # D inv = Degree ^-0.5 
        d_mat_inv_sqrt = sp.diags(d_inv_sqrt) #substitution of the 1D array degree in a 2D matrix diagonals
        normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo() # norm= D^-0.5 * adj * D^-0.5
        normalized_adj = normalized_adj.astype(np.float32) 
        return normalized_adj
     
    ''' def weight_variable_glorot(input_dim, output_dim, name=""):
        init_range = np.sqrt(6.0 / (input_dim + output_dim))
        initial = tf.random_uniform([input_dim, output_dim], minval=-init_range, maxval=init_range, dtype=tf.float32)
        return tf.Variable(initial,name=name) '''
    
    def sparse_to_tuple(self,mx):
        mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose() #coordinate stacking row and column wise and transpose
        L = tf.SparseTensor(coords, mx.data, mx.shape) # mx.shape= (156,156)
        #print('shape ---->',mx.shape)
        return tf.sparse.reorder(L) #row major ordering
        
    def init_state(self,batch_size):       
        state = tf.zeros(shape=[batch_size, self._num_nodes*self._num_units], dtype=tf.float32)
        return state  
               
    @staticmethod
    def _concat(x, x_):
        x_ = tf.expand_dims(x_, 0)
        return tf.concat([x, x_], axis=0)   
    @property
    def state_size(self):
        return self._nodes * self._units

    @property
    def output_size(self):
        return self._units

    def __call__(self, inputs, state, scope=None):

        with tf.compat.v1.variable_scope(scope or "tgcn",reuse=tf.compat.v1.AUTO_REUSE):
            with tf.compat.v1.variable_scope("gates",reuse=tf.compat.v1.AUTO_REUSE):  
                value = tf.nn.sigmoid(
                    self._gc(inputs, state, 2 * self._units, bias=1.0, scope=scope))
                r, u = tf.split(value=value, num_or_size_splits=2, axis=1)
            with tf.compat.v1.variable_scope("candidate",reuse=tf.compat.v1.AUTO_REUSE):
                r_state = r * state
                c = self._act(self._gc(inputs, r_state, self._units, scope=scope))
            new_h = u * state + (1 - u) * c
        return new_h, new_h


    def _gc(self, inputs, state, output_size, bias=0.0, scope=None):
        ## inputs:(-1,num_nodes)
        inputs = tf.expand_dims(inputs, 2)
        ## state:(batch,num_node,gru_units)
        state = tf.reshape(state, (-1, self._nodes, self._units))#(-1,156,64)
        ## concat
        x_s = tf.concat([inputs, state], axis=2)
        input_size = x_s.get_shape()[2]
        ## (num_node,input_size,-1)
        x0 = tf.transpose(x_s, perm=[1, 2, 0])  
        x0 = tf.reshape(x0, shape=[self._nodes, -1])
        
        scope = tf.compat.v1.get_variable_scope()
        with tf.compat.v1.variable_scope(scope):
            for m in self._adj:
                x1 = tf.compat.v1.sparse_tensor_dense_matmul(m, x0)
#                print(x1)
            x = tf.reshape(x1, shape=[self._nodes, input_size,-1])
            x = tf.transpose(x,perm=[2,0,1])
            x = tf.reshape(x, shape=[-1, input_size])
            initializer = tf.initializers.GlorotUniform()
            weights = tf.Variable(initializer(shape=(input_size, output_size)))
            x = tf.matmul(x, weights)  # (batch_size * self._nodes, output_size)
            biases = tf.compat.v1.get_variable(
                "t_biases", [output_size], initializer=tf.constant_initializer(bias))
            x = tf.nn.bias_add(x, biases)
            x = tf.reshape(x, shape=[-1, self._nodes, output_size])
            x = tf.reshape(x, shape=[-1, self._nodes * output_size])
        return x

In [ ]:
def TSTGCN(_X, _weights, _biases):
    ###
    cell_1 = tstgcnCell(num_units, adj, num_nodes=num_nodes)
    cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([cell_1], state_is_tuple=True)
    _X = tf.unstack(_X, axis=1)
    #rnncell = tf.keras.layers.RNN(cell, return_state=True)
    #outputs, states= rnncell(_X)
    outputs, states=tf.compat.v1.nn.static_rnn(cell, _X, dtype=tf.float32)
    m = []
    for i in outputs:
        o = tf.reshape(i,shape=[-1,num_nodes,num_units])
        o = tf.reshape(o,shape=[-1,num_units])
        m.append(o)
    last_output = m[-1]
    output = tf.matmul(last_output, _weights['out']) + _biases['out']
    output = tf.reshape(output,shape=[-1,num_nodes,pre_len])
    output = tf.transpose(output, perm=[0,2,1])
    output = tf.reshape(output, shape=[-1,num_nodes])
    return output, m, states

In [ ]:
tf.compat.v1.disable_eager_execution()
t_inputs = tf.compat.v1.placeholder(tf.float32, shape=[None, seq_len, num_nodes])
t_labels = tf.compat.v1.placeholder(tf.float32, shape=[None, pre_len, num_nodes])
t_weights = {
    'out': tf.Variable(tf.random.normal([num_units, pre_len], mean=1.0), name='weight_o')}
t_biases = {
    'out': tf.Variable(tf.random.normal([pre_len]),name='bias_o')}
print(type(inputs))
tpred,ttts,ttto = TSTGCN(t_inputs, t_weights, t_biases)

In [ ]:
ty_pred = tpred

In [ ]:
lambda_loss = 0.0015
Lreg = lambda_loss * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.compat.v1.trainable_variables())
t_label = tf.reshape(t_labels, [-1,num_nodes])

In [ ]:
t_loss = tf.reduce_mean(tf.nn.l2_loss(ty_pred-t_label) + Lreg)

In [ ]:
t_error = tf.sqrt(tf.reduce_mean(tf.square(ty_pred-t_label)))

In [ ]:
lr=0.001
t_optimizer=opt.minimize(t_loss)

In [ ]:
variables = tf.compat.v1.global_variables()
training_epoch=1000


In [ ]:
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

In [ ]:
def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    r2 = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return rmse, mae, 1-F_norm, r2, var

In [ ]:
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_acc,test_r2,test_var,test_pred = [],[],[],[],[],[],[]

In [ ]:
def acc(a,b):  
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    train_acc=1-F_norm
    return train_acc

In [ ]:
for epoch in range(training_epoch):
    print("Epoch ", epoch)
    for m in range(totalbatch):
        mini_batch = trainX[m * batch_size : (m+1) * batch_size]
        mini_label = trainY[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)
        batch_rmse.append(rmse1 * max_value)
        train_label=np.reshape(mini_label,[-1,num_nodes])
     #print(mini_label.shape,train_output.shape) (32, 1, 156) (32, 156)
     # Test completely at every epoch
    print("Accuracy ----> ", evaluation(train_label,train_output)[2])
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:testX, labels:testY})
    #train_label=np.reshape(trainY,[-1,num_nodes])
    #train_acc=acc(train_label,train_output)
    test_label = np.reshape(testY,[-1,num_nodes])
    rmse, mae, acc, r2_score, var_score = evaluation(test_label, test_output)
    test_label1 = test_label * max_value#Inverse normalization
    test_output1 = test_output * max_value
    test_loss.append(loss2)
    test_rmse.append(rmse * max_value)
    test_mae.append(mae * max_value)
    test_acc.append(acc)
    test_r2.append(r2_score)
    test_var.append(var_score)
    test_pred.append(test_output1)
    #print(mini_label.shape,train_output.shape)
    print('Iter:{}'.format(epoch),
          'train_rmse:{:.4}'.format(batch_rmse[-1]),
          'test_loss:{:.4}'.format(loss2),
          'test_rmse:{:.4}'.format(rmse),
          'test_acc:{:.4}'.format(acc))
    
        
time_end = time.time()
print('Time taken : ',time_end-time_start,'s')

In [ ]:
b = int(len(batch_rmse)/totalbatch)
batch_rmse1 = [i for i in batch_rmse]
train_rmse = [(sum(batch_rmse1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
batch_loss1 = [i for i in batch_loss]
train_loss = [(sum(batch_loss1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
index = test_rmse.index(np.min(test_rmse))
test_result = test_pred[index]
var = pd.DataFrame(test_result)
#plot_result(test_result,test_label1,path)
#plot_error(train_rmse,train_loss,test_rmse,test_acc,test_mae,path)

In [ ]:
print("Testing:")
print('min_rmse:%r'%(np.min(test_rmse)),
      'min_mae:%r'%(test_mae[index]),
      'max_acc:%r'%(test_acc[index]),
      'r2:%r'%(test_r2[index]),
      'var:%r'%test_var[index])

In [ ]:
print("Training:")
rmse, mae, acc, r2_score, var_score = evaluation(train_label,train_output)
print('min_rmse:%r'%(rmse),
      'min_mae:%r'%(mae),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score))

In [ ]:
#inverse normalisation
print('min_rmse:%r'%(rmse*max_value),
      'min_mae:%r'%(mae*max_value),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score))